In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from teste_datas import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot

from bokeh import palettes

In [2]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'
fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/decan4.IEC'

In [4]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)
    
counts = spec2.spCounts
print(len(counts))

ret = cwt_complete_analysis(counts)
# ret = gaussian_complete_analysis(counts)

(xs, y0s, ar1, lblimag1, num_feat_imag1, ridges, xpks, ypks,
 ind_ypks, retang_indicess, peaks_masks, cwtpks, nscpks) = ret

Erro de arquivo IEC !!!


AttributeError: 'Spec' object has no attribute 'spCounts'

In [ ]:
# 2020-05-15 Espectros Mariana PR.

In [ ]:
# Desativado:

In [ ]:
(nscpk_ma, cwtpk_ma, pospk_ma) = peaks_masks
valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask | ~pospk_ma)

In [ ]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# Bokeh graph w/ peaks in all scales

# Gráfico de ridges com qualidade dos picos pelas cores e tamanhos:

# Navy 2:     pontos dos ridgest
# Vermelho 8: picos válidos
# Laranja 8:  inválido, aparece em poucas escalas  
# Verde 6:    inválido, muito pequeno (pelo coeficiente cwt)
# Ciano 3:    inválido, forma ruim (cwt perto dos extremos do ridge)

bokridges1 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   # x_axis_label='channels',
   y_axis_label='scale',
   title=fn2,
)
bokridges1.circle(ar1[:,0], ar1[:,1], color="navy",size=2)
bokridges1.circle(xpks[valid_peaks], ypks[valid_peaks], color="red",size=8)
bokridges1.circle(xpks[nscpk_ma.mask], ypks[nscpk_ma.mask], color="orange",size=8)
bokridges1.circle(xpks[cwtpk_ma.mask], ypks[cwtpk_ma.mask], color="green",size=6)
bokridges1.circle(xpks[~pospk_ma], ypks[~pospk_ma], color="cyan",size=3)
bokridges2 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[1.0e1, 1.0e5],
   x_axis_label='channels', y_axis_label='counts'
)
bokridges2.line(xs, y0s, color="green")
# https://docs.bokeh.org/en/latest/docs/reference/layouts.html
p = gridplot([bokridges1, bokridges2], ncols=1)
show(p)

In [ ]:
valid_xpks = np.asarray(xpks)[valid_peaks]
valid_xpks

In [ ]:
valid_ypks = np.asarray(ypks)[valid_peaks]
valid_ypks

In [ ]:
valid_cwtpks = np.asarray(cwtpks)[valid_peaks]
valid_cwtpks

In [ ]:
xs

In [ ]:
y0s

In [ ]:
step2 = cwt_calc_peaks(valid_xpks, valid_ypks, valid_cwtpks, xs, y0s)

In [ ]:
xpks[pospk_ma]


In [ ]:
(azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
           baseline_spectrum, bl_median_smoother) = step2

In [ ]:
azes

In [ ]:
len(azes)

In [ ]:
bezes

In [ ]:
centrpk

In [ ]:
#azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
#           baseline_spectrum, bl_median_smoother) = step2

In [ ]:
bezes[0]

In [ ]:
gau_areas

In [ ]:
# 2020-02-24
# Tudo em progresso! Agora:
# - Fazer os gr'aficos
# - fazer em v'arias passadas o c'alculo at'e n~ao haver picos na linha-base.

In [ ]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0],
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
bokpksallscl.line(xs, peak_constructed_spectrum, color="red")
bokpksallscl.line(xs, bl_median_smoother, color="orange")
bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [ ]:
# Bokeh graph w/ peaks in all scales
bok2 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="green")
bok2.line(xs, peak_constructed_spectrum, color="red")
# bokpksallscl.line(xs, bl_median_smoother, color="cyan")
# bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bok2)

In [ ]:
# 2020-02-25 Segunda passada:
# 2020-05-15 Por enquanto deixarei desativada; mas posso fazer manualmente, bastando rodar
#            as duas células seguintes e repetir o trecho que se inicia logo após
#            a determinação de valid_peaks.

In [ ]:
# 2020-05-17 Continuando para cada espectro. Este é Decan4.iec

In [ ]:
spec2.samDesc

In [ ]:
print(spec2.spLVTime)

In [ ]:
print(spec2.spRLTime)

In [ ]:
print(spec2.diahora)